In [1]:
import numpy as np
import pandas as pd
import nltk
from sklearn import feature_extraction, linear_model, model_selection, preprocessing

In [2]:
train_df = pd.read_csv(r"C:\Users\Hp\Documents\Python_scripts\NLP with disaster tweets/train.csv")
test_df = pd.read_csv(r"C:\Users\Hp\Documents\Python_scripts\NLP with disaster tweets/test.csv")

In [3]:
train_df.head(2)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1


In [4]:
test_df.head(2)

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."


In [5]:
train_df[train_df['target'] == 1]['text'].values[0]

'Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all'

In [6]:
train_df[train_df['target'] == 0]['text'].values[0]

"What's up man?"

In [7]:
nltk.download('punkt')     ## required for tokenization
nltk.download('averaged_perceptron_tagger')     ## brequired for pos tagging
from nltk.tokenize import RegexpTokenizer
from nltk import word_tokenize, pos_tag

train_df_tokenize = [RegexpTokenizer("[\w']+").tokenize(word) for word in train_df['text']]
test_df_tokenize = [RegexpTokenizer("[\w']+").tokenize(word) for word in test_df['text']]
train_df_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Hp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Hp\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


[['Our',
  'Deeds',
  'are',
  'the',
  'Reason',
  'of',
  'this',
  'earthquake',
  'May',
  'ALLAH',
  'Forgive',
  'us',
  'all'],
 ['Forest', 'fire', 'near', 'La', 'Ronge', 'Sask', 'Canada'],
 ['All',
  'residents',
  'asked',
  'to',
  "'shelter",
  'in',
  "place'",
  'are',
  'being',
  'notified',
  'by',
  'officers',
  'No',
  'other',
  'evacuation',
  'or',
  'shelter',
  'in',
  'place',
  'orders',
  'are',
  'expected'],
 ['13',
  '000',
  'people',
  'receive',
  'wildfires',
  'evacuation',
  'orders',
  'in',
  'California'],
 ['Just',
  'got',
  'sent',
  'this',
  'photo',
  'from',
  'Ruby',
  'Alaska',
  'as',
  'smoke',
  'from',
  'wildfires',
  'pours',
  'into',
  'a',
  'school'],
 ['RockyFire',
  'Update',
  'California',
  'Hwy',
  '20',
  'closed',
  'in',
  'both',
  'directions',
  'due',
  'to',
  'Lake',
  'County',
  'fire',
  'CAfire',
  'wildfires'],
 ['flood',
  'disaster',
  'Heavy',
  'rain',
  'causes',
  'flash',
  'flooding',
  'of',
  'stree

In [8]:
nltk.download('wordnet')
from nltk.stem.wordnet import WordNetLemmatizer
import re, string

def clean_tweets(text, stop_words = ()):
    
    lemmatizer = WordNetLemmatizer()
    cleaned_tweet = []
    for word, tag in pos_tag(text):
    
        word = re.sub('((http|https)\:\/\/)?[a-zA-Z0-9\.\/\?\:@\-_=#]+\.([a-zA-Z]){2,6}([a-zA-Z0-9\.\&\/\?\:@\-_=#])*','', word)
        word = re.sub("@(a-zA-Z0-9_)", '',word)
        word = re.sub('\w*\d\w*', '', word)
        word = re.sub('http|co|û_|amp', '', word)
    
        if tag.startswith('NN'):
            pos = 'n'
        if tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'
            
        word = lemmatizer.lemmatize(word,pos)
        if len(word) > 0 and word not in string.punctuation and word.lower() not in stop_words:
            cleaned_tweet.append(word.lower())
            
    return cleaned_tweet      

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Hp\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [9]:
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

train_tokens = []
test_tokens = []

for token in train_df_tokenize:
    train_tokens.append(clean_tweets(token, stop_words))
    
for token in test_df_tokenize:
    test_tokens.append(clean_tweets(token, stop_words))
    
train_tokens

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Hp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


[['deeds', 'reason', 'earthquake', 'may', 'allah', 'forgive', 'us'],
 ['forest', 'fire', 'near', 'la', 'ronge', 'sask', 'canada'],
 ['residents',
  'ask',
  "'shelter",
  "place'",
  'notify',
  'officers',
  'evacuation',
  'shelter',
  'place',
  'orders',
  'expect'],
 ['people', 'receive', 'wildfires', 'evacuation', 'orders', 'california'],
 ['get',
  'send',
  'photo',
  'ruby',
  'alaska',
  'smoke',
  'wildfires',
  'pour',
  'school'],
 ['rockyfire',
  'update',
  'california',
  'hwy',
  'close',
  'directions',
  'due',
  'lake',
  'county',
  'fire',
  'cafire',
  'wildfires'],
 ['flood',
  'disaster',
  'heavy',
  'rain',
  'cause',
  'flash',
  'flooding',
  'streets',
  'manitou',
  'colorado',
  'springs',
  'areas'],
 ["i'm", 'top', 'hill', 'see', 'fire', 'woods'],
 ["there's",
  'emergency',
  'evacuation',
  'happen',
  'building',
  'across',
  'street'],
 ["i'm", 'afraid', 'tornado', 'ming', 'area'],
 ['three', 'people', 'die', 'heat', 'wave', 'far'],
 ['haha',
  's

In [10]:
def list_to_string(token):
    space = " "
    s = []
    for text in token:
        s.append(space.join(text))
    return s

train_tokens_str = list_to_string(train_tokens)
test_tokens_str = list_to_string(test_tokens)

In [11]:

train_df['clean_text'] = train_tokens_str
test_df['clean_text'] = test_tokens_str
train_df.head()


,id,keyword,location,text,target,clean_text
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,deeds reason earthquake may allah forgive us
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,forest fire near la ronge sask canada
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,residents ask 'shelter place' notify officers ...
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,people receive wildfires evacuation orders cal...
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,get send photo ruby alaska smoke wildfires pou...


In [13]:
count_vectorizer = feature_extraction.text.CountVectorizer()

example_train_vectors = count_vectorizer.fit_transform(train_df['clean_text'][0:5])

print(example_train_vectors[0].todense().shape)
print(example_train_vectors[0].todense())

(1, 35)
[[0 1 0 0 0 1 1 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0]]


In [15]:
train_vectors = count_vectorizer.fit_transform(train_df["clean_text"])
test_vectors = count_vectorizer.transform(test_df["clean_text"])

In [17]:
clf = linear_model.RidgeClassifier()

scores = model_selection.cross_val_score(clf, train_vectors, train_df["target"], cv=5, scoring="f1")
scores

array([0.59871442, 0.47058824, 0.56454121, 0.52268603, 0.63279446])

In [18]:
clf.fit(train_vectors, train_df["target"])

RidgeClassifier(alpha=1.0, class_weight=None, copy_X=True, fit_intercept=True,
                max_iter=None, normalize=False, random_state=None,
                solver='auto', tol=0.001)

In [21]:
sample_submission = pd.read_csv(r"C:\Users\Hp\Documents\Python_scripts\NLP with disaster tweets/sample_submission.csv")

In [22]:
sample_submission["target"] = clf.predict(test_vectors)

In [26]:
sample_submission.head()

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1


In [24]:
sample_submission.to_csv("submission.csv", index=False)